### Import libraries and create DataFrame from csv 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("QB_Data.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Josh Allen (BUF),385,579,66.5,4306,7.4,29,18,24,111,524,15,17,410.9,24.2
1,2,Jalen Hurts (PHI),352,538,65.4,3858,7.2,23,15,36,157,605,15,17,371.9,21.9
2,3,Dak Prescott (DAL),410,590,69.5,4516,7.7,36,9,39,55,242,2,17,352.0,20.7
3,4,Lamar Jackson (BAL),307,457,67.2,3678,8.0,24,7,37,148,821,5,16,338.2,21.1
4,5,Jordan Love (GB),372,579,64.2,4159,7.2,32,11,30,50,247,4,17,330.1,19.4
5,6,Brock Purdy (SF),308,444,69.4,4280,9.6,31,11,28,39,144,2,16,306.6,19.2
6,7,Jared Goff (DET),407,605,67.3,4575,7.6,30,12,30,32,21,2,17,302.9,17.8
7,8,Patrick Mahomes II (KC),401,597,67.2,4183,7.0,27,14,27,75,389,0,16,294.1,18.4
8,9,Tua Tagovailoa (MIA),388,560,69.3,4624,8.3,29,14,29,35,74,0,17,284.4,16.7
9,10,Baker Mayfield (TB),364,566,64.3,4044,7.1,28,10,40,62,163,1,17,284.0,16.7


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [7]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['PCT', 'Y/A', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,PCT,Y/A,G,FPTS,FPTS/G,CMP/game,ATT/game,YDS/game,TD/game,INT/game,SACKS/game,ATT.1/game,YDS.1/game,TD.1/game
0,1,Josh Allen (BUF),66.5,7.4,17,410.9,24.2,22.6,34.1,253.3,1.7,1.1,1.4,6.5,30.8,0.9
1,2,Jalen Hurts (PHI),65.4,7.2,17,371.9,21.9,20.7,31.6,226.9,1.4,0.9,2.1,9.2,35.6,0.9
2,3,Dak Prescott (DAL),69.5,7.7,17,352.0,20.7,24.1,34.7,265.6,2.1,0.5,2.3,3.2,14.2,0.1
3,4,Lamar Jackson (BAL),67.2,8.0,16,338.2,21.1,19.2,28.6,229.9,1.5,0.4,2.3,9.2,51.3,0.3
4,5,Jordan Love (GB),64.2,7.2,17,330.1,19.4,21.9,34.1,244.6,1.9,0.6,1.8,2.9,14.5,0.2
5,6,Brock Purdy (SF),69.4,9.6,16,306.6,19.2,19.2,27.8,267.5,1.9,0.7,1.8,2.4,9.0,0.1
6,7,Jared Goff (DET),67.3,7.6,17,302.9,17.8,23.9,35.6,269.1,1.8,0.7,1.8,1.9,1.2,0.1
7,8,Patrick Mahomes II (KC),67.2,7.0,16,294.1,18.4,25.1,37.3,261.4,1.7,0.9,1.7,4.7,24.3,0.0
8,9,Tua Tagovailoa (MIA),69.3,8.3,17,284.4,16.7,22.8,32.9,272.0,1.7,0.8,1.7,2.1,4.4,0.0
9,10,Baker Mayfield (TB),64.3,7.1,17,284.0,16.7,21.4,33.3,237.9,1.6,0.6,2.4,3.6,9.6,0.1


### Calculate the correlations for the final stats across different conditions

In [8]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
PCT,0.72,-0.03,0.33,0.47,0.37
Y/A,0.54,-0.08,0.42,0.48,0.34
CMP/game,0.92,0.88,0.69,0.30,0.69
ATT/game,0.90,0.86,0.63,0.15,0.64
YDS/game,0.94,0.91,0.78,0.54,0.79
TD/game,0.93,0.91,0.83,0.68,0.84
INT/game,0.52,0.35,0.11,0.12,0.28
SACKS/game,0.65,0.50,-0.16,-0.39,0.15
ATT.1/game,0.63,0.59,0.41,0.49,0.53
YDS.1/game,0.59,0.54,0.37,0.42,0.48


### Assign the weights for the final stats

In [9]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
PCT,0.37,0.14,0.07
Y/A,0.34,0.12,0.07
CMP/game,0.69,0.48,0.09
ATT/game,0.64,0.41,0.09
YDS/game,0.79,0.63,0.14
TD/game,0.84,0.70,0.15
INT/game,0.28,0.08,0.07
SACKS/game,0.15,0.02,0.07
ATT.1/game,0.53,0.28,0.08
YDS.1/game,0.48,0.23,0.08


### Multiply the assigned weights to the final stats and calculate the score

In [10]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,PCT_weighted,Y/A_weighted,CMP/game_weighted,ATT/game_weighted,YDS/game_weighted,TD/game_weighted,ATT.1/game_weighted,YDS.1/game_weighted,TD.1/game_weighted
0,1,Josh Allen (BUF),24.2,4.8,0.5,2.1,3.1,36.6,0.3,0.5,2.4,0.1
1,2,Jalen Hurts (PHI),21.9,4.8,0.5,2.0,2.8,32.7,0.2,0.8,2.8,0.1
2,3,Dak Prescott (DAL),20.7,5.0,0.5,2.3,3.1,38.3,0.3,0.3,1.1,0.0
3,4,Lamar Jackson (BAL),21.1,4.9,0.6,1.8,2.6,33.2,0.2,0.8,4.0,0.0
4,5,Jordan Love (GB),19.4,4.7,0.5,2.1,3.1,35.3,0.3,0.2,1.1,0.0
5,6,Brock Purdy (SF),19.2,5.0,0.7,1.8,2.5,38.6,0.3,0.2,0.7,0.0
6,7,Jared Goff (DET),17.8,4.9,0.5,2.3,3.2,38.8,0.3,0.2,0.1,0.0
7,8,Patrick Mahomes II (KC),18.4,4.9,0.5,2.4,3.3,37.7,0.3,0.4,1.9,0.0
8,9,Tua Tagovailoa (MIA),16.7,5.0,0.6,2.2,2.9,39.2,0.3,0.2,0.3,0.0
9,10,Baker Mayfield (TB),16.7,4.7,0.5,2.0,3.0,34.3,0.2,0.3,0.8,0.0


### Model training

In [11]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 1.3096654593575319
Test Set MSE: 1.0001399535301032
Random Forest Test Set MSE: 0.1739134900967354


/tmp/ipykernel_11354/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [12]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("QB_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,PCT,Y/A,G,FPTS,FPTS/G,CMP/game,YDS/game,TD/game,INT/game,SACKS/game,ATT.1/game
Rank,,,,,,,,,,,,,,,
1,1,Josh Allen (BUF),10.00,0,66.5,7.4,17,410.9,24.2,22.6,253.3,1.7,1.1,1.4,6.5
24,2,Kirk Cousins (ATL),9.47,22,69.5,7.5,8,154.9,19.4,27.0,291.4,2.2,0.6,2.1,1.8
3,3,Dak Prescott (DAL),9.41,0,69.5,7.7,17,352.0,20.7,24.1,265.6,2.1,0.5,2.3,3.2
2,4,Jalen Hurts (PHI),9.25,-2,65.4,7.2,17,371.9,21.9,20.7,226.9,1.4,0.9,2.1,9.2
4,5,Lamar Jackson (BAL),9.23,-1,67.2,8.0,16,338.2,21.1,19.2,229.9,1.5,0.4,2.3,9.2
6,6,Brock Purdy (SF),9.04,0,69.4,9.6,16,306.6,19.2,19.2,267.5,1.9,0.7,1.8,2.4
8,7,Patrick Mahomes II (KC),9.04,1,67.2,7.0,16,294.1,18.4,25.1,261.4,1.7,0.9,1.7,4.7
32,8,Joe Flacco (IND),8.87,24,60.3,7.9,5,108.9,21.8,24.6,323.2,2.6,1.6,1.6,1.8
7,9,Jared Goff (DET),8.83,-2,67.3,7.6,17,302.9,17.8,23.9,269.1,1.8,0.7,1.8,1.9
